In [24]:
# Load env variables and create client
import base64
from dotenv import load_dotenv
from anthropic import Anthropic
from anthropic.types import Message
from pathlib import Path
from base64 import standard_b64encode, standard_b64decode

load_dotenv()

client = Anthropic()
model = "claude-3-7-sonnet-latest"
head_cost = 1024
messages = []

In [25]:
# Helper functions
from anthropic.types import Message


def add_user_message(messages, message):
    user_message = {
        "role": "user",
        "content": message.content if isinstance(message, Message) else message,
    }
    messages.append(user_message)


def add_assistant_message(messages, message):
    assistant_message = {
        "role": "assistant",
        "content": message.content if isinstance(message, Message) else message,
    }
    messages.append(assistant_message)


def chat(
    messages,
    system=None,
    temperature=1.0,
    stop_sequences=[],
    tools=None,
    thinking=False,
    thinking_budget=head_cost,
):
    params = {
        "model": model,
        "max_tokens": 4000,
        "messages": messages,
        "temperature": temperature,
        "stop_sequences": stop_sequences,
    }

    if thinking:
        params["thinking"] = {
            "type": "enabled",
            "budget_tokens": thinking_budget,
        }

    if tools:
        params["tools"] = tools

    if system:
        params["system"] = system

    message = client.messages.create(**params)
    return message


def text_from_message(message):
    return "\n".join(
        [block.text for block in message.content if block.type == "text"]
    )

In [26]:
# Fire risk assessment prompt
prompt = """
Analyze the attached satellite image of a property with these specific steps:

1. Residence identification: Locate the primary residence on the property by looking for:
   - The largest roofed structure 
   - Typical residential features (driveway connection, regular geometry)
   - Distinction from other structures (garages, sheds, pools)
   Describe the residence's location relative to property boundaries and other features.

2. Tree overhang analysis: Examine all trees near the primary residence:
   - Identify any trees whose canopy extends directly over any portion of the roof
   - Estimate the percentage of roof covered by overhanging branches (0-25%, 25-50%, 50-75%, 75-100%)
   - Note particularly dense areas of overhang

3. Fire risk assessment: For any overhanging trees, evaluate:
   - Potential wildfire vulnerability (ember catch points, continuous fuel paths to structure)
   - Proximity to chimneys, vents, or other roof openings if visible
   - Areas where branches create a "bridge" between wildland vegetation and the structure
   
4. Defensible space identification: Assess the property's overall vegetative structure:
   - Identify if trees connect to form a continuous canopy over or near the home
   - Note any obvious fuel ladders (vegetation that can carry fire from ground to tree to roof)

5. Fire risk rating: Based on your analysis, assign a Fire Risk Rating from 1-4:
   - Rating 1 (Low Risk): No tree branches overhanging the roof, good defensible space around the structure
   - Rating 2 (Moderate Risk): Minimal overhang (<25% of roof), some separation between tree canopies
   - Rating 3 (High Risk): Significant overhang (25-50% of roof), connected tree canopies, multiple points of vulnerability
   - Rating 4 (Severe Risk): Extensive overhang (>50% of roof), dense vegetation against structure, numerous ember catch points, limited defensible space

For each item above (1-5), write one sentence summarizing your findings, with your final response being the numeric Fire Risk Rating (1-4) with a brief justification.
"""

In [27]:
# TODO: Read image data, feed into Claude
img_dir = Path.cwd()
img_list = [i for i in img_dir.iterdir() if i.suffix == ".png"]
#[print(i.suffix) for i in img_dir.iterdir()]
[print(i) for i in img_list]
img_bytes = []
for i in img_list:
   with i.open("rb") as f:
      img_bytes.append(standard_b64encode(f.read()).decode("utf-8"))

/home/jx-creator/Projects/console/tutorials/claude_features/prop3.png
/home/jx-creator/Projects/console/tutorials/claude_features/prop7.png
/home/jx-creator/Projects/console/tutorials/claude_features/prop2.png
/home/jx-creator/Projects/console/tutorials/claude_features/prop5.png
/home/jx-creator/Projects/console/tutorials/claude_features/prop1.png
/home/jx-creator/Projects/console/tutorials/claude_features/prop4.png
/home/jx-creator/Projects/console/tutorials/claude_features/prop6.png


In [30]:
add_user_message(messages, [
   # Image Code Block Here
   {
      "type": "image",
      "source": {
         "type": "base64",
         "media_type": "image/png",
         "data": img_bytes[0]
      }
   },
   # Text Code Block Here
   {
      "type": "text",
      "text": "What do you see in this image?"
   }
])

response = chat(messages, system=prompt)
response.content[0].text

"Based on the satellite image:\n\n1. The primary residence is clearly identifiable as the two-story structure with a dark roof and light-colored walls, located in the central portion of the property with a driveway connecting to a road on the left side.\n\n2. There appears to be no significant tree overhang directly over the residence's roof, with trees being positioned around the property perimeter but maintaining distance from the main structure.\n\n3. The fire risk from overhanging trees is minimal as there are no apparent canopies extending over the roof structure, reducing potential ember catch points.\n\n4. The property demonstrates good defensible space with trees primarily positioned at the property boundaries and adequate separation between vegetation clusters and the main residence.\n\n5. Fire Risk Rating: 1 (Low Risk) - The property features excellent defensible space with no tree branches overhanging the roof, good distance between the residence and surrounding vegetation, 